In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from corpy.get_data.get_data import data_download

data_download()

Downloading...
From: https://drive.google.com/u/0/uc?id=11ozVs6zByFjs9viD3VIIP6qKFgjZwv9E&export=download
To: /home/roger/code/xkeeja/corpy-assessment/screws.zip
100%|██████████| 195M/195M [00:07<00:00, 27.3MB/s] 


Unzipping file...
Data successfully unzipped to data/archive.
Deleting zip file...
Deleted zip file.
